In [1]:
import pandas as pd
import os

train = pd.read_csv('data/train_df.csv', encoding='euc-kr')
test = pd.read_csv('data/test_df.csv', encoding='euc-kr')
submission = pd.read_csv('data/sample_submission.csv')
population = pd.read_csv('data/202001_202001_주민등록인구및세대현황_월간.csv', encoding='euc-kr')
space_code = pd.concat([pd.read_csv('data/TC_NU_SPG_50_METER/'+ file) for file in os.listdir('data/TC_NU_SPG_50_METER/')])

In [2]:
train_data, train_target = train.drop(['index', 'INVC_CONT'],axis=1), train['INVC_CONT']
test_data = test.drop(['index'],axis=1)

data = pd.concat([train_data, test_data])
data

,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM
0,1129000014045300,5011000220046300,패션의류,상의
1,1135000009051200,5011000178037300,생활/건강,반려동물
2,1135000030093100,5011000265091400,패션의류,기타패션의류
3,1154500002014200,5011000315087400,식품,농산물
4,1165000021008300,5011000177051200,식품,가공식품
...,...,...,...,...
4635,5013000858004400,4725000719072200,식품,농산물
4636,5013000870018300,2826000106075300,식품,농산물
4637,5013000897086300,4311100034004300,식품,농산물
4638,5013000902065100,4145000013011200,식품,농산물


In [3]:
space_code = space_code[space_code['격자공간고유번호'].isin(data['SEND_SPG_INNB'].unique()) | space_code['격자공간고유번호'].isin(data['REC_SPG_INNB'].unique())]
space_code = space_code.drop(['시군구코드','시군구명'], axis=1)
space_code

,격자공간고유번호,격자공간명
1052,1150000003064100,다사3854
2716,1150000007080100,다사3853
2793,1150000007099200,다사3853
2934,1150000008034300,다사3953
2986,1150000008047300,다사3953
...,...,...
833734,5013000954034300,나나8469
833766,5013000954042300,나나8469
834976,5013000957045100,나나8769
835085,5013000957072200,나나8769


In [4]:
population['시군구코드'] = population['행정구역'].apply(lambda x: int(x[-11:-6]))
population = population.drop(['행정구역'], axis=1)
population

,2020년01월_총인구수,2020년01월_세대수,2020년01월_세대당 인구,2020년01월_남자 인구수,2020년01월_여자 인구수,2020년01월_남여 비율,시군구코드
0,"9,733,509","4,334,918",2.25,"4,745,088","4,988,421",0.95,11000
1,"151,215","73,925",2.05,"73,688","77,527",0.95,11110
2,"126,201","62,814",2.01,"61,946","64,255",0.96,11140
3,"229,385","110,457",2.08,"110,701","118,684",0.93,11170
4,"300,410","135,732",2.21,"147,020","153,390",0.96,11200
...,...,...,...,...,...,...,...
273,"62,049","29,508",2.10,"30,275","31,774",0.95,48880
274,"45,054","24,155",1.87,"21,590","23,464",0.92,48890
275,"670,749","293,362",2.29,"337,231","333,518",1.01,50000
276,"489,202","209,614",2.33,"245,389","243,813",1.01,50110


In [5]:
data = data.merge(space_code, how='left', left_on='SEND_SPG_INNB', right_on='격자공간고유번호')
data = data.merge(space_code, how='left', left_on='REC_SPG_INNB', right_on='격자공간고유번호')

data = data.drop(['격자공간고유번호_x', '격자공간고유번호_y'], axis=1)
data['SEND_시군구코드'] = data['SEND_SPG_INNB'].apply(lambda x: int(str(x)[:5]))
data['REC_시군구코드'] = data['REC_SPG_INNB'].apply(lambda x: int(str(x)[:5]))

data = data.merge(population, how='left', left_on='SEND_시군구코드', right_on='시군구코드')
data = data.merge(population, how='left', left_on='REC_시군구코드', right_on='시군구코드')

data = data.drop(['시군구코드_x','시군구코드_y'], axis=1)
data.columns = ['SEND_SPG_INNB', 'REC_SPG_INNB', 'DL_GD_LCLS_NM', 'DL_GD_MCLS_NM',
       'SEND_격자공간명', 'REC_격자공간명', 'SEND_시군구코드', 'REC_시군구코드',
       'SEND_총인구수', 'SEND_세대수', 'SEND_세대당인구', 'SEND_남자인구수', 'SEND_여자인구수', 'SEND_남여비율',
       'REC_총인구수', 'REC_세대수', 'REC_세대당인구', 'REC_남자인구수','REC_여자인구수', 'REC_남여비율']

In [6]:
data

,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,SEND_격자공간명,REC_격자공간명,SEND_시군구코드,REC_시군구코드,SEND_총인구수,SEND_세대수,SEND_세대당인구,SEND_남자인구수,SEND_여자인구수,SEND_남여비율,REC_총인구수,REC_세대수,REC_세대당인구,REC_남자인구수,REC_여자인구수,REC_남여비율
0,1129000014045300,5011000220046300,패션의류,상의,다사6156,다다1502,11290,50110,"442,471","192,809",2.29,"214,247","228,224",0.94,"489,202","209,614",2.33,"245,389","243,813",1.01
1,1135000009051200,5011000178037300,생활/건강,반려동물,다사6163,다다1903,11350,50110,"532,662","217,054",2.45,"257,746","274,916",0.94,"489,202","209,614",2.33,"245,389","243,813",1.01
2,1135000030093100,5011000265091400,패션의류,기타패션의류,다사6158,다다1001,11350,50110,"532,662","217,054",2.45,"257,746","274,916",0.94,"489,202","209,614",2.33,"245,389","243,813",1.01
3,1154500002014200,5011000315087400,식품,농산물,다사4541,다다0600,11545,50110,"233,066","111,000",2.10,"118,299","114,767",1.03,"489,202","209,614",2.33,"245,389","243,813",1.01
4,1165000021008300,5011000177051200,식품,가공식품,다사5942,다다1803,11650,50110,"431,050","173,388",2.49,"206,112","224,938",0.92,"489,202","209,614",2.33,"245,389","243,813",1.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36635,5013000858004400,4725000719072200,식품,농산물,다나0473,라바5923,50130,47250,"181,547","83,748",2.17,"91,842","89,705",1.02,"99,814","48,527",2.06,"48,984","50,830",0.96
36636,5013000870018300,2826000106075300,식품,농산물,다나1673,다사2548,50130,28260,"181,547","83,748",2.17,"91,842","89,705",1.02,"544,608","218,156",2.50,"274,954","269,654",1.02
36637,5013000897086300,4311100034004300,식품,농산물,나나9472,다바9950,50130,43111,"181,547","83,748",2.17,"91,842","89,705",1.02,"179,626","74,371",2.42,"89,423","90,203",0.99
36638,5013000902065100,4145000013011200,식품,농산물,다나0472,다사7250,50130,41450,"181,547","83,748",2.17,"91,842","89,705",1.02,"273,587","116,659",2.35,"137,533","136,054",1.01


In [7]:
lcls_dict = dict()
for lcls in train['DL_GD_LCLS_NM'].unique():
    lcls_dict[lcls] = train.groupby('DL_GD_LCLS_NM').mean().loc[lcls, 'INVC_CONT']

mcls_dict = dict()
for mcls in train['DL_GD_MCLS_NM'].unique():
    mcls_dict[mcls] = train.groupby('DL_GD_MCLS_NM').mean().loc[mcls, 'INVC_CONT']

In [8]:
data['mean_lcls'] = data['DL_GD_LCLS_NM'].map(lcls_dict)
data['mean_mcls'] = data['DL_GD_MCLS_NM'].map(mcls_dict)

In [9]:
for col in data.columns:
    if col not in ['DL_GD_LCLS_NM','DL_GD_MCLS_NM','SEND_격자공간명','REC_격자공간명'] and data[col].dtype == 'object':
        data[col] = data[col].str.replace(',','').astype(float)

data = data.fillna(data.mean())

C:\Users\KDH\AppData\Local\Temp/ipykernel_15376/982961410.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data = data.fillna(data.mean())


In [10]:
data.isnull().sum()

SEND_SPG_INNB    0
REC_SPG_INNB     0
DL_GD_LCLS_NM    0
DL_GD_MCLS_NM    0
SEND_격자공간명       0
REC_격자공간명        0
SEND_시군구코드       0
REC_시군구코드        0
SEND_총인구수        0
SEND_세대수         0
SEND_세대당인구       0
SEND_남자인구수       0
SEND_여자인구수       0
SEND_남여비율        0
REC_총인구수         0
REC_세대수          0
REC_세대당인구        0
REC_남자인구수        0
REC_여자인구수        0
REC_남여비율         0
mean_lcls        0
mean_mcls        0
dtype: int64

In [11]:
train_data, test_data = data.iloc[:-len(test_data)], data.iloc[-len(test_data):]
train_data['INVC_CONT'] = train_target

C:\Users\KDH\AppData\Local\Temp/ipykernel_15376/3505830822.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['INVC_CONT'] = train_target


In [12]:
train_data

,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,SEND_격자공간명,REC_격자공간명,SEND_시군구코드,REC_시군구코드,SEND_총인구수,SEND_세대수,...,SEND_남여비율,REC_총인구수,REC_세대수,REC_세대당인구,REC_남자인구수,REC_여자인구수,REC_남여비율,mean_lcls,mean_mcls,INVC_CONT
0,1129000014045300,5011000220046300,패션의류,상의,다사6156,다다1502,11290,50110,442471.0,192809.0,...,0.94,489202.000000,209614.000000,2.330000,245389.000000,243813.000000,1.010000,6.678694,3.672897,3
1,1135000009051200,5011000178037300,생활/건강,반려동물,다사6163,다다1903,11350,50110,532662.0,217054.0,...,0.94,489202.000000,209614.000000,2.330000,245389.000000,243813.000000,1.010000,5.407921,4.444134,3
2,1135000030093100,5011000265091400,패션의류,기타패션의류,다사6158,다다1001,11350,50110,532662.0,217054.0,...,0.94,489202.000000,209614.000000,2.330000,245389.000000,243813.000000,1.010000,6.678694,10.375479,9
3,1154500002014200,5011000315087400,식품,농산물,다사4541,다다0600,11545,50110,233066.0,111000.0,...,1.03,489202.000000,209614.000000,2.330000,245389.000000,243813.000000,1.010000,4.658195,4.297401,10
4,1165000021008300,5011000177051200,식품,가공식품,다사5942,다다1803,11650,50110,431050.0,173388.0,...,0.92,489202.000000,209614.000000,2.330000,245389.000000,243813.000000,1.010000,4.658195,6.807151,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31995,5011001060063300,2635000026053400,식품,농산물,나나8483,마라4789,50110,26350,489202.0,209614.0,...,1.01,406312.000000,168851.000000,2.410000,196037.000000,210275.000000,0.930000,4.658195,4.297401,6
31996,5011001095042400,1168000017002200,식품,기타식품,나나8581,다사6044,50110,11680,489202.0,209614.0,...,1.01,545938.000000,233363.000000,2.340000,261190.000000,284748.000000,0.920000,4.658195,6.130548,5
31997,5011001108036200,4119700008012100,식품,농산물,나나8380,다사3841,50110,41197,489202.0,209614.0,...,1.01,350729.658507,149491.193251,2.312961,174980.441026,175749.217481,0.999579,4.658195,4.297401,9
31998,5011001115011400,1132000015085100,식품,농산물,나나7779,다사5861,50110,11320,489202.0,209614.0,...,1.01,332753.000000,138509.000000,2.400000,162701.000000,170052.000000,0.960000,4.658195,4.297401,3


In [13]:
test_data

,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,SEND_격자공간명,REC_격자공간명,SEND_시군구코드,REC_시군구코드,SEND_총인구수,SEND_세대수,...,SEND_여자인구수,SEND_남여비율,REC_총인구수,REC_세대수,REC_세대당인구,REC_남자인구수,REC_여자인구수,REC_남여비율,mean_lcls,mean_mcls
32000,5013000043028400,1165000021097200,식품,농산물,다나4294,다사5942,50130,11650,181547.0,83748.0,...,89705.0,1.02,431050.0,173388.0,2.49,206112.0,224938.0,0.92,4.658195,4.297401
32001,5013000044016100,1154500002066400,식품,농산물,다나4394,다사4541,50130,11545,181547.0,83748.0,...,89705.0,1.02,233066.0,111000.0,2.10,118299.0,114767.0,1.03,4.658195,4.297401
32002,5013000205030200,4139000102013200,식품,농산물,다나4187,다사3029,50130,41390,181547.0,83748.0,...,89705.0,1.02,475396.0,195728.0,2.43,248693.0,226703.0,1.10,4.658195,4.297401
32003,5013000205030200,4221000040093400,식품,농산물,다나4187,라아9121,50130,42210,181547.0,83748.0,...,89705.0,1.02,81840.0,38723.0,2.11,40470.0,41370.0,0.98,4.658195,4.297401
32004,5013000268011400,2726000004017100,식품,농산물,다나3285,마마0564,50130,27260,181547.0,83748.0,...,89705.0,1.02,428770.0,167410.0,2.56,208071.0,220699.0,0.94,4.658195,4.297401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36635,5013000858004400,4725000719072200,식품,농산물,다나0473,라바5923,50130,47250,181547.0,83748.0,...,89705.0,1.02,99814.0,48527.0,2.06,48984.0,50830.0,0.96,4.658195,4.297401
36636,5013000870018300,2826000106075300,식품,농산물,다나1673,다사2548,50130,28260,181547.0,83748.0,...,89705.0,1.02,544608.0,218156.0,2.50,274954.0,269654.0,1.02,4.658195,4.297401
36637,5013000897086300,4311100034004300,식품,농산물,나나9472,다바9950,50130,43111,181547.0,83748.0,...,89705.0,1.02,179626.0,74371.0,2.42,89423.0,90203.0,0.99,4.658195,4.297401
36638,5013000902065100,4145000013011200,식품,농산물,다나0472,다사7250,50130,41450,181547.0,83748.0,...,89705.0,1.02,273587.0,116659.0,2.35,137533.0,136054.0,1.01,4.658195,4.297401


In [14]:
train_data.to_csv('data/train_updated.csv', index=False)
test_data.to_csv('data/test_updated.csv', index=False)